## Aufgabe: Erste Gedanken zu synthetischer Zeichnung
Verfasst von: Alex Wagner\
Im Zeitraum von: 16.10.2024 - 20.10.2024

___

### 1. Zum Gedanken: Greifen der Teile durch den Roboter

Zur Erinnerung: Angehoben werden die Teile vom Roboter durch einen mit Druckluft versehenen Sauggreifer.

Aufgrund der teilweise komplexen Form der Teile, ist eine anspruchvolle Herangehensweise zur Bestimmung des besten Punktes, an dem der Roboter die Teile greifen kann, notwendig.

Ein Vorschlag wäre deshalb eine Bestimmung des bestmöglichen Punktes, zum Greifen durch den Roboter, durch eine euklidische Distanztransformation ([distanceTransform()](https://docs.opencv.org/3.4/d7/d1b/group__imgproc__misc.html#ga8a0b7fdfcb7a13dde018988ba3a43042)), welche in OpenCV realisiert werden kann.

___

Zur Erklärung (an einem NumPy-Array):

Angenommen, die Teile wären von der Oberfläche auf der sie liegen zu unterscheiden, und würden nicht aufeinander gestapelt liegen:

Dann wäre es ein Leichtes, das Bild eines Teiles mit Hilfe einer Maske in OpenCV in ein Schwarz-Weiß-Bild zu transformieren, auf welchem das Teil in weiß, und der Hintergrund in schwarz abgebildet werden.\
(In OpenCV unterscheiden sich intern die weißen und schwarzen Pixel, indem die weißen Pixel als Einsen und die schwarzen Pixel als Nullen gespeichert werden.)

Folgendes NumPy-Array soll sinnbildlich ein Teil auf einer Oberfläche darstellen, wobei jeweils die Einsen die weißen Pixel, also die Pixel, auf denen das Teil zu sehen ist, und die Nullen die schwarzen Pixel, also die Pixel, auf denen der Hintergrund zu sehen ist, repräsentieren:

In [1]:
import numpy as np

arr = np.array([[0, 0, 0, 0, 0, 0, 0],
                [0, 1, 1, 1, 1, 1, 0],
                [0, 1, 1, 1, 1, 1, 0],
                [0, 1, 1, 1, 1, 1, 0],
                [0, 1, 1, 1, 1, 1, 0],
                [0, 1, 1, 1, 1, 1, 0],
                [0, 0, 0, 0, 0, 0, 0]])
print(arr)

[[0 0 0 0 0 0 0]
 [0 1 1 1 1 1 0]
 [0 1 1 1 1 1 0]
 [0 1 1 1 1 1 0]
 [0 1 1 1 1 1 0]
 [0 1 1 1 1 1 0]
 [0 0 0 0 0 0 0]]


Eine euklidische Distanztransformation (euclidean distance transform) bewirkt Folgendes:

Für jedes Pixel:
   1. Bestimme die euklidische Distanz (kürzester Abstand) zur nächsten Null\
     (Bei den Nullen kommt jeweils Null raus, da die Distanz zu sich selber, also zur nächsten Null, gleich Null ist)
   2. Trage die euklidische Distanz anstelle des Wertes Null bzw. Eins ein

In [2]:
from scipy import ndimage

print(ndimage.distance_transform_edt(arr))

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 2. 2. 2. 1. 0.]
 [0. 1. 2. 3. 2. 1. 0.]
 [0. 1. 2. 2. 2. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


Der Punkt, mit dem höchsten Wert, ist der Punkt mit der höchsten Distanz zum Rand des Teiles.\
Demzufolge wäre das die beste Stelle zum Greifen des Teiles durch den Roboter, um mit jeder möglichen Form des Teiles, ohne Vorkenntnis über dessen Aussehen, zurechtzukommen.
___

### 2. Zum Gedanken: Wissen über die Position des Teiles im Raum ###

Wenn der Roboter ein Teil an einer, anhand der euklidischen Distanztransformation bestimmten, Stelle greift, wäre es für den Roboter gut zu wissen, "wie" und "wo" sich das Teil im Raum befindet.

Durch Erkennung einer oder mehrerer Ecken mittels Eckenerkennung in OpenCV ([Harris Corner Detection](https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html), [cornerHarris()](https://docs.opencv.org/3.4/dd/d1a/group__imgproc__feature.html#gac1fc3598018010880e370e2f709b4345)), hätte der Roboter "Referenzpunkte", anhand welcher er genau wüsste, wo sich eine oder mehrere Ecken, und somit Grenzen des Teiles, in Referenz zu der Stelle, an welcher der Roboter das Teil greift, im Raum befinden.